## Titanic

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras import models, layers, Sequential, initializers
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

2022-11-02 12:33:00.159244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Train Data

In [5]:
df_train = pd.read_csv("./titanic/train.csv")

df_train.drop(columns=["PassengerId", "Cabin"], inplace=True)

df_train["Embarked"].fillna(df_train["Embarked"].unique()[0], inplace=True)

df_train['Name'] = df_train['Name'].str.split(', ').str[1].str.split('. ').str[0]
df_train['Master'] = (df_train['Name']=='Master')
df_train=df_train.drop(columns='Name')
df_train=df_train.drop(columns='Ticket')

# age_new 컬럼 생성.
df_train.loc[df_train.Age >= 50, "age_new"] = "old"
df_train.loc[(df_train.Age < 50) & (df_train.Age>=10), "age_new"] = "young"
df_train.loc[df_train.Age < 10, "age_new"] = "baby"

# 인코딩
og_columns = df_train.columns[(df_train.dtypes=='O')|(df_train.dtypes=='category')|(df_train.dtypes=='bool')]
for i in og_columns : 
    globals()[f'df_train_{i}_encoder'] = LabelEncoder()
    globals()[f'df_train_{i}_encoder'].fit(df_train[i])
    df_train[i] = globals()[f'df_train_{i}_encoder'].transform(df_train[i])
    
# 연령의 결측치 해결
Age_md = df_train.groupby(['Pclass', 'Sex']).Age.agg(['median'])
df_train.loc[(df_train['Sex'] ==  0) & (df_train['Pclass'] == 1) & (df_train.Age.isna()), "Age"] = Age_md.loc[1, 0][0]
df_train.loc[(df_train['Sex'] == 0) & (df_train['Pclass'] == 2) & (df_train.Age.isna()), "Age"] = Age_md.loc[2, 0][0]
df_train.loc[(df_train['Sex'] == 0) & (df_train['Pclass'] == 3) & (df_train.Age.isna()), "Age"] = Age_md.loc[3, 0][0]
df_train.loc[(df_train['Sex'] == 1) & (df_train['Pclass'] == 1) & (df_train.Age.isna()), "Age"] = Age_md.loc[1, 1][0]
df_train.loc[(df_train['Sex'] == 1) & (df_train['Pclass'] == 2) & (df_train.Age.isna()), "Age"] = Age_md.loc[2, 1][0]
df_train.loc[(df_train['Sex'] == 1) & (df_train['Pclass'] == 3) & (df_train.Age.isna()), "Age"] = Age_md.loc[3, 1][0]

df_train["family"] = df_train.SibSp + df_train.Parch
df_train.drop(columns=["SibSp", "Parch"], inplace=True)

# X, Y , test, train 분리
X = df_train.drop(columns='Survived')
y = df_train['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)


## Test Data

In [6]:
df_test = pd.read_csv('./titanic/test.csv')

df_test.drop(columns=["PassengerId", "Cabin"], inplace=True)

df_test['Name'] = df_test['Name'].str.split(', ').str[1].str.split('. ').str[0]
df_test['Master']=(df_test['Name']=='Master')
df_test=df_test.drop(columns='Name')
df_test=df_test.drop(columns='Ticket')

# 연령의 결측치 해결
# fare 결측치 해결
age_md = df_test.groupby(['Pclass', 'Sex']).Age.agg(['mean'])
df_test.loc[(df_test['Sex'] == 'male') & (df_test['Pclass'] == 1) & (df_test.Age.isna()), 'Age'] = age_md.loc[1, 'male'][0]
df_test.loc[(df_test['Sex'] == 'male') & (df_test['Pclass'] == 2) & (df_test.Age.isna()), 'Age'] = age_md.loc[2, 'male'][0]
df_test.loc[(df_test['Sex'] == 'male') & (df_test['Pclass'] == 3) & (df_test.Age.isna()), 'Age'] = age_md.loc[3, 'male'][0]
df_test.loc[(df_test['Sex'] == 'female') & (df_test['Pclass'] == 1) & (df_test.Age.isna()), 'Age'] = age_md.loc[1, 'female'][0]
df_test.loc[(df_test['Sex'] == 'female') & (df_test['Pclass'] == 2) & (df_test.Age.isna()), 'Age'] = age_md.loc[2, 'female'][0]
df_test.loc[(df_test['Sex'] == 'female') & (df_test['Pclass'] == 3) & (df_test.Age.isna()), 'Age'] = age_md.loc[3, 'female'][0]
df_test.Fare.fillna(df_test["Fare"].mean(), inplace=True)

# age_new 생성
df_test.loc[df_test.Age >= 50, "age_new"] = "old"
df_test.loc[(df_test.Age < 50) & (df_test.Age>=10), "age_new"] = "young"
df_test.loc[df_test.Age < 10, "age_new"] = "baby"

df_test["family"] = df_test.SibSp + df_test.Parch
df_test.drop(columns=["SibSp", "Parch"], inplace=True)

og_columns = df_test.columns[(df_test.dtypes=='O')|(df_test.dtypes=='category')|(df_test.dtypes=='bool')]
for i in og_columns : 
    globals()[f'df_test{i}_encoder'] = LabelEncoder()
    globals()[f'df_test{i}_encoder'].fit(df_test[i])
    df_test[i] = globals()[f'df_test{i}_encoder'].transform(df_test[i])

## Answer Data

In [31]:
df_answer = pd.read_csv("./titanic/answer_tit.csv")
df_answer = df_answer.Survived

<span style="color:blue; font-size:30px">Deep Learning</span>

## Create Model

In [5]:
inputs = Input(shape=(8,))
n = Dense(256, activation="elu")(inputs)
n = Dropout(0.5)(n)
n = BatchNormalization()(n)
n = Dense(128, activation="elu")(n)
n = Dropout(0.5)(n)
n = BatchNormalization()(n)
n = Dense(64, activation="elu")(n)
n = Dropout(0.5)(n)
n = BatchNormalization()(n)
n = Dense(32, activation="elu")(n)
n = Dropout(0.5)(n)
n = BatchNormalization()(n)
n = Dense(16, activation="elu")(n)
n = Dropout(0.5)(n)
n = BatchNormalization()(n)
n = Dense(1, activation="sigmoid")(n)

model = Model(inputs=inputs, outputs=n)
model.compile(loss="mse", 
              optimizer="adam", 
              metrics=["acc"])

checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, save_weights_only=True,
    monitor='val_accuracy', mode='max', save_best_only=True)

model.fit(X, y, epochs=300, verbose=0, validation_split=0.2, callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)

2022-11-01 17:23:13.257148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(df_test, df_answer, verbose=2)

print(f"test loss : {test_loss}, test acc : {test_acc}")


## Fit & Predict Model
## Appraisal Model

In [ ]:
model.fit(X, y, epochs=300, validation_split=0.2)

In [ ]:
test_loss, test_acc = model.evaluate(df_test, df_answer, verbose=2)

print(f"test loss : {test_loss}, test acc : {test_acc}")

## HyperParameter in Model

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import keras_tuner
from tensorflow import keras

def build_model(hp):
    ip = Input(shape=(X.shape[1],))
    n = BatchNormalization()(ip)
    n = Dense(hp.Choice('units', [8, 16, 32, 64, 128, 256]),
              activation='elu')(n)
    n = Dropout(0.5)(n)
    n = BatchNormalization()(n)
    n = Dense(hp.Choice('units', [8, 16, 32, 64, 128, 256]),
              activation='elu')(n)
    n = BatchNormalization()(n)
    n = Dense(hp.Choice('units', [8, 16, 32, 64, 128, 256]),
              activation='elu')(n)
    n = Dropout(0.5)(n)
    n = BatchNormalization()(n)
    n = Dense(hp.Choice('units', [8, 16, 32, 64, 128, 256]),
              activation='elu')(n)
    n = Dense(1, activation='sigmoid')(n)
    model = Model(inputs=ip, outputs=n)
    model.compile(loss='mse', optimizer="adam", metrics='accuracy')
    return model

In [ ]:
tuner4 = keras_tuner.RandomSearch(
    build_model, objective='val_accuracy', max_trials=5, directory='./tuner4')

In [ ]:
tuner4.search(X, y, epochs=1000, validation_split=0.1)

In [ ]:
best_model3 = tuner4.get_best_models()[0]

In [ ]:
# best is 3
best_model3.evaluate(df_test, df_answer)

<span style="color:blue; font-size:30px">Machine Learning</span>

## VotingClassifier
### RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

In [6]:
rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, max_depth=5, 
                                 criterion='entropy', max_features=None,)
et_clf = ExtraTreesClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, max_depth=10, 
                              bootstrap=True, random_state=10, max_samples=1.0)
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=1000, 
                            max_samples=1.0, bootstrap=False, n_jobs=-1,
                            bootstrap_features=True, max_features=0.5, oob_score=False)
voting_model = VotingClassifier(estimators=[('RandomForestClassifier', rnd_clf), 
                                            ('BaggingClassifier', bag_clf),
                                            ('ExtraTreesClassifier', et_clf)], 
                                voting='soft', n_jobs=-1)


voting_model = VotingClassifier(estimators=[('RandomForestClassifier', rnd_clf), 
                                            ('BaggingClassifier', bag_clf),
                                            ('ExtraTreesClassifier', et_clf)], 
                                voting='soft', n_jobs=-1)

voting_model.fit(X, y)
pred = voting_model.predict(df_test)
accuracy_score(df_answer, pred)


0.7918660287081339

## Stacking Ensemble

### Set Model

In [36]:
from sklearn.ensemble import HistGradientBoostingClassifier

svm = SVC(random_state=0, probability=True)
rf = RandomForestClassifier(n_estimators=100, random_state=0)
lr = LogisticRegression()
hgb = HistGradientBoostingClassifier()

### fit model

In [37]:
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)
hgb.fit(X_train, y_train)

/Users/werther/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


HistGradientBoostingClassifier()

### predict proba

In [38]:
svm_proba = svm.predict_proba(X_train)[:, 1]
rf_proba = rf.predict_proba(X_train)[:, 1]
lr_proba = lr.predict_proba(X_train)[:, 1]
hgb_proba = hgb.predict_proba(X_train)[:, 1]

### make meta dataframe
> ###### 메타 데이터 프레임을 확인하여 모델의 성능을 평가

In [39]:
new_data = np.array([svm_proba, rf_proba, lr_proba, hgb_proba])
meta_df = pd.DataFrame(new_data.T)
meta_df.columns = ['svm_proba', 'rf_proba', 'lr_proba', 'hgb_proba']
meta_df['y'] = y_train.values
meta_df

,svm_proba,rf_proba,lr_proba,hgb_proba,y
0,0.310098,0.020000,0.357313,0.035353,0
1,0.300966,0.160000,0.108251,0.132029,0
2,0.301469,0.010000,0.051128,0.012123,0
3,0.301465,0.000000,0.071592,0.002521,0
4,0.380672,0.997500,0.924366,0.987342,1
...,...,...,...,...,...
707,0.313118,0.910000,0.712041,0.878661,1
708,0.302266,0.013048,0.138779,0.122317,0
709,0.313652,0.080000,0.240711,0.032616,0
710,0.302451,0.780000,0.081737,0.713913,1


###### 4개의 모델의 평균 값을 확인

In [73]:
meta_df.mean(axis=1)

0      0.180691
1      0.175311
2      0.093680
3      0.093895
4      0.822470
         ...   
707    0.703455
708    0.144102
709    0.166745
710    0.469525
711    0.094003
Length: 712, dtype: float64

## stacking and fit

In [40]:
meta_dt = DecisionTreeClassifier(ccp_alpha=0.01)
meta_dt.fit(meta_df.drop('y', axis=1), meta_df.y)

DecisionTreeClassifier(ccp_alpha=0.01)

In [19]:
meta_df

,svm_pred,rf_pred,lr_pred,lgbm_pred,y
0,0.310098,0.020000,0.357313,0.035353,0
1,0.300966,0.160000,0.108251,0.132029,0
2,0.301469,0.010000,0.051128,0.012123,0
3,0.301465,0.000000,0.071592,0.002521,0
4,0.380672,0.997500,0.924366,0.987342,1
...,...,...,...,...,...
707,0.313118,0.910000,0.712041,0.878661,1
708,0.302266,0.013048,0.138779,0.122317,0
709,0.313652,0.080000,0.240711,0.032616,0
710,0.302451,0.780000,0.081737,0.713913,1


### last fit model & evaluate model

In [41]:
svm_pred = svm.predict(X_test)
rf_pred = rf.predict(X_test)
lr_pred = lr.predict(X_test)
hgb_pred = hgb.predict(X_test)

new_data = np.array([svm_pred, rf_pred, lr_pred, hgb_pred]).T

In [ ]:
lgbm.fit(new_data, df_answer)
lgbm_pred = lgbm.predict(df_test)
accuracy_score(df_test, lgbm_pred)

In [26]:
from sklearn.ensemble import StackingClassifier

stk_clf = StackingClassifier([svm_pred, rf_pred, lr_pred, hgb_pred],
    final_estimator=DecisionTreeClassifier(ccp_alpha=0.01),
    cv=5,
    stack_method='auto',
    n_jobs=4)

In [45]:
stk_clf.predict(X_test)

AttributeError: 'StackingClassifier' object has no attribute 'final_estimator_'

In [29]:
y_train.shape

(712,)

In [32]:
stk_clf.predict(df_test, df_answer)

TypeError: predict() takes 2 positional arguments but 3 were given